AZURE PROJECT
USING MEDALIAN ARCHITECTURE 

In [0]:
storageAccountName = "projectstorageaccounts"
storageAccountAccessKey = 'd1U58MYmaZd8BcT5zMyJrVbMIFGFCu3qDFunqXRWMNCK/4CFu7gaDNLLjNrQNuZkMG2qfq5/S22B+AStvjhwbg=='
blobContainerName = "project"
mountPoint = "/mnt/data/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
      #extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

mount succeeded!


In [0]:
dbutils.widgets.dropdown('w_sel','reviews',['reviews','products','sales'],'list_of_files')

In [0]:
dbutils.widgets.get('w_sel')

Out[56]: 'reviews'

In [0]:
#pipeline parameters
mnt_pnt= '/mnt/data'
file_path= '*.txt'
raw_file_path= mnt_pnt+'/raw/'+file_path
#bronze
raw_db_name='bronze'
tbl_name=dbutils.widgets.get('w_sel')
db_bronze_path=mnt_pnt+'/'+raw_db_name
raw_table_name=tbl_name+'_stg'
raw_table_path=mnt_pnt+'/'+raw_db_name+'/'+raw_table_name
#silver
silver_db_name='silver'
silver_db_path=mnt_pnt+'/'+silver_db_name
silver_table_path=mnt_pnt+'/'+silver_db_name+'/'+tbl_name
print(silver_table_path)
#archive path
archive_path=mnt_pnt+'/archive'


/mnt/data/silver/reviews


In [0]:
dbutils.fs.ls(mnt_pnt)

Out[58]: [FileInfo(path='dbfs:/mnt/data/archive/', name='archive/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/data/bronze/', name='bronze/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/data/raw/', name='raw/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/data/silver/', name='silver/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.mkdirs('/mnt/data/archive')

Out[59]: True

In [0]:
dbutils.fs.mkdirs('/mnt/data/raw')

Out[60]: True

In [0]:
df=spark.read.format('csv').option("header",True).load(raw_file_path)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1834024260580303>:1
----> 1 df=spark.read.format('csv').option("header",True).load(raw_file_path)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:302, in DataFrameReader.load(self, path, format, schema, **options)
    300 self.options(**options)
    301 if isinstance(path, str):
--> 302     return self._df(self._jreader.load(path))
    303 elif path is not None:
    304     if type(path) != list:

File /databricks/spark/python/lib/py4j-0.10.9.5-src


RAW TO BRONZE

In [0]:
qry=f'''
                create database if not exists {raw_db_name}
                    location '{db_bronze_path}'
      '''          
spark.sql(qry)    

In [0]:
from datetime import datetime
from pyspark.sql.functions import *


In [0]:
dt = datetime.now().strftime("%d-%m-%y")
df = df.withColumn("etldate",lit(dt))

In [0]:
qry=f'''
               create table if not exists {raw_db_name}.{raw_table_name}
               using delta
               location '{raw_table_path}'
     '''
spark.sql(qry)                

In [0]:
df.write.format('delta').mode('overwrite').option('mergeschema',True).save(raw_table_path)

BRONZE TO SILVER


In [0]:
qry=f'''
           create database if not exists silver
           location '{silver_db_path}'
      '''
spark.sql(qry)           

In [0]:
qry=f'''
        create table if not exists {silver_db_name}.{tbl_name}
        using delta
        location '{silver_table_path}'
      '''
spark.sql(qry)   

In [0]:
df.write.format('delta').mode('append').option('mergeschema',True).save(silver_table_path)

In [0]:
file_lst=dbutils.fs.ls('/mnt/data/raw/')
dbutils.fs.mkdirs(archive_path)
for file in file_lst:
    target_folder=archive_path+'/'+dt
    dbutils.fs.mkdirs(target_folder)
    dbutils.fs.mv(file.path,target_folder)

In [0]:
if tbl_name == 'reviews':
    df.write.format('delta').mode('overwrite').option('mergeschema',True).save(raw_table_path)
elif tbl_name =='products':
    df.write.format('delta').mode('append').option('mergeschema',True).save(raw_table_path)
elif tbl_name =='categories':
    qry =f'''merge into silver.categories c
             using bronze.categories_stg cs
             on c.category_id =cs.category_id
             when matched then update set *
             when not matched then insert *
           '''
spark.sql(qry)                     
    

In [0]:
dbutils.fs.unmount(mnt_pnt)